In [ ]:
from pyspark.sql.functions import broadcast, split, lit, col, split


matchesBucketed = spark.read.option("header", "true") \
                        .option("inferSchema", "true") \
                        .csv("/home/iceberg/data/matches.csv")
matchDetailsBucketed =  spark.read.option("header", "true") \
                        .option("inferSchema", "true") \
                        .csv("/home/iceberg/data/match_details.csv")


spark.sql("""DROP TABLE IF EXISTS bootcamp.matches_bucketed""")
bucketedDDL = """
 CREATE TABLE IF NOT EXISTS bootcamp.matches_bucketed (
    match_id STRING,
     is_team_game BOOLEAN,
     playlist_id STRING,
     completion_date TIMESTAMP
 )
 USING iceberg
 PARTITIONED BY (completion_date, bucket(16, match_id));
 """
spark.sql(bucketedDDL)

matchesBucketed.select(
     col("match_id"), col("is_team_game"), col("playlist_id"), col("completion_date")
     ) \
     .write.mode("append")  \
     .partitionBy("completion_date") \
     .bucketBy(16, "match_id").saveAsTable("bootcamp.matches_bucketed")


bucketedDetailsDDL = """
 CREATE TABLE IF NOT EXISTS bootcamp.match_details_bucketed (
     match_id STRING,
     player_gamertag STRING,
     player_total_kills INTEGER,
     player_total_deaths INTEGER
 )
 USING iceberg
 PARTITIONED BY (bucket(16, match_id));
 """
spark.sql(bucketedDetailsDDL)

# // matchDetailsBucketed.select(
# //     $"match_id", $"player_gamertag", $"player_total_kills", $"player_total_deaths")
# //     .write.mode("append")
# //   .bucketBy(16, "match_id").saveAsTable("bootcamp.match_details_bucketed")

# spark.conf.set("spark.sql.autoBroadcastJoinThreshold", "-1")

matchesBucketed.createOrReplaceTempView("matches")
matchDetailsBucketed.createOrReplaceTempView("match_details")

spark.sql("""
    SELECT * FROM bootcamp.match_details_bucketed mdb JOIN bootcamp.matches_bucketed md 
    ON mdb.match_id = md.match_id
    AND md.completion_date = DATE('2016-01-01')
        
""").explain()


spark.sql("""
    SELECT * FROM match_details mdb JOIN matches md ON mdb.match_id = md.match_id
        
""").explain()


spark.conf.set("spark.sql.autoBroadcastJoinThreshold", "1000000000000")

broadcastFromThreshold = matches.alias("m").join(matchDetails.alias("md"), col("m.match_id") == col("md.match_id")) \
   .select(col("m.completion_date"), col("md.player_gamertag"),  col("md.player_total_kills")) \
   .take(5)

explicitBroadcast = matches.alias("m").join(broadcast(matchDetails).alias("md"), col("m.match_id") == col("md.match_id")) \
   .select(col("md.*"), split(col("completion_date"), " ").getItem(0).alias("ds"))

bucketedValues = matchDetailsBucketed.alias("mdb").join(matchesBucketed.alias("mb"), col("mb.match_id") == col("mdb.match_id")).explain().take(5)

values = matchDetailsBucketed.alias("m").join(matchesBucketed.alias("md"), col("m.match_id") == col("md.match_id")).explain()

explicitBroadcast.write.mode("overwrite").insertInto("bootcamp.match_details_bucketed")

matches.withColumn("ds", split(col("completion_date"), " ").getItem(0)).write.mode("overwrite").insertInto("matches_bucketed")

spark.sql(bucketedSQL)



SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".                
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.
